In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch 10. Iris 다양한 DNN</font><b>

In [2]:
import os # 모델 저장할 폴더 확인 / 폴더 생성
import numpy as np
import pandas as pd # get_dummies, crosstab
import seaborn as sns # iris 데이터(df) 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코딩
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [24]:
# 1. 데이터셋
iris = datasets.load_iris() # target 분리까지 되어 있음
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, 
                                                    train_size=0.8, 
                                                    stratify=iris_y #층화 추출
                                                    )
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [25]:
train_y

array([2, 0, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 2, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 2, 2, 0, 1, 2, 1, 0, 2, 0, 2, 2, 2, 0, 1, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 1, 2, 2, 1,
       1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       0, 2, 2, 1, 0, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1, 1,
       0, 2, 0, 2, 2, 2, 1, 0, 1, 2])

In [26]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(64, 'relu'),
    Dense(128, 'relu'),
    Dense(50, 'relu'),
    Dense(30, 'relu'),
    Dense(3, 'softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                320       
                                                                 
 dense_16 (Dense)            (None, 128)               8320      
                                                                 
 dense_17 (Dense)            (None, 50)                6450      
                                                                 
 dense_18 (Dense)            (None, 30)                1530      
                                                                 
 dense_19 (Dense)            (None, 3)                 93        
                                                                 
Total params: 16,713
Trainable params: 16,713
Non-trainable params: 0
_________________________________________________________________


In [27]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model_save_folder = './model10/'
if not os.path.exists(model_save_folder):
    os.mkdir(model_save_folder)
file = model_save_folder + 'iris2-{epoch:03d}-loss{val_loss:.4f}-acc{val_accuracy:.4f}.h5'
checkpoint = ModelCheckpoint(
                filepath = file,
                monitor='val_accuracy',
                save_best_only=True,
                mode = 'max',
                verbose=0)
earlyStopping = EarlyStopping(monitor='val_loss', patience=20)

In [28]:
# 4. 모델 학습
hist = model.fit(train_X, train_y, 
                 epochs=200, 
                 validation_split=0.1)

Epoch 1/200
4/4 [==============================] - 0s 49ms/step - loss: 1.0438 - accuracy: 0.6019 - val_loss: 0.8914 - val_accuracy: 0.6667
Epoch 2/200
4/4 [==============================] - 0s 6ms/step - loss: 0.8878 - accuracy: 0.6667 - val_loss: 0.7914 - val_accuracy: 0.6667
Epoch 3/200
4/4 [==============================] - 0s 7ms/step - loss: 0.8005 - accuracy: 0.7963 - val_loss: 0.7381 - val_accuracy: 0.9167
Epoch 4/200
4/4 [==============================] - 0s 8ms/step - loss: 0.7124 - accuracy: 0.7963 - val_loss: 0.6629 - val_accuracy: 0.6667
Epoch 5/200
4/4 [==============================] - 0s 18ms/step - loss: 0.6392 - accuracy: 0.6852 - val_loss: 0.5986 - val_accuracy: 0.6667
Epoch 6/200
4/4 [==============================] - 0s 8ms/step - loss: 0.5701 - accuracy: 0.7407 - val_loss: 0.5406 - val_accuracy: 0.8333
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 0.5153 - accuracy: 0.9259 - val_loss: 0.4996 - val_accuracy: 1.0000
Epoch 8/200
4/4 [=======

4/4 [==============================] - 0s 11ms/step - loss: 0.0906 - accuracy: 0.9815 - val_loss: 0.0239 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0945 - accuracy: 0.9722 - val_loss: 0.0174 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0773 - accuracy: 0.9722 - val_loss: 0.0150 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0873 - accuracy: 0.9815 - val_loss: 0.0172 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0877 - accuracy: 0.9630 - val_loss: 0.0150 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0711 - accuracy: 0.9815 - val_loss: 0.0203 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0921 - accuracy: 0.9630 - val_loss: 0.0146 - val_accuracy: 1.0000
Epoch 66/200
4/4 [=============

4/4 [==============================] - 0s 5ms/step - loss: 0.0657 - accuracy: 0.9815 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 118/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0727 - accuracy: 0.9815 - val_loss: 0.0102 - val_accuracy: 1.0000
Epoch 119/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0656 - accuracy: 0.9815 - val_loss: 0.0098 - val_accuracy: 1.0000
Epoch 120/200
4/4 [==============================] - 0s 12ms/step - loss: 0.0633 - accuracy: 0.9815 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 121/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0627 - accuracy: 0.9907 - val_loss: 0.0086 - val_accuracy: 1.0000
Epoch 122/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0615 - accuracy: 0.9907 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 123/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0628 - accuracy: 0.9907 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 124/200
4/4 [=======

4/4 [==============================] - 0s 6ms/step - loss: 0.0654 - accuracy: 0.9815 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0626 - accuracy: 0.9815 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0608 - accuracy: 0.9815 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0547 - accuracy: 0.9815 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 179/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0647 - accuracy: 0.9815 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 180/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0636 - accuracy: 0.9815 - val_loss: 0.0133 - val_accuracy: 1.0000
Epoch 181/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0940 - accuracy: 0.9630 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 182/200
4/4 [=====

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동된다.

In [30]:
from sklearn.neural_network import MLPClassifier

In [29]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [31]:
model = MLPClassifier(hidden_layer_sizes=(64, 128, 50), #hidden layer의 units 수 
                      activation='relu', # Classifier 객체라, softmax 안 넣어도 기능함
                      solver='adam', # sgd 등의 optimizer
                      batch_size=40,
                      max_iter=1000, # 학습 최대 횟수(==epochs)
                      early_stopping=True, # 조기 종료 활성화
                      n_iter_no_change=10, #earlyStopping의 patience와 같은 기능 (10회 이상 발전 못하면 중단)
                      validation_fraction=0.1, # 검증셋 비율
                      warm_start=False, # coldstart. 다시 실행했을 때 학습을 잇지 말고 새 것부터 하자. 
                     )

In [32]:
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50), max_iter=1000)

In [ ]:
# model.evaluate(test_X, test_y, batch_size=100, verbose=0)
# MLPClassifer는 evaluate가 안 되네?

In [33]:
model.score(test_X, test_y)

0.9666666666666667

In [34]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [36]:
input_data=[[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [37]:
# 교차표
# 실제값 : test_y
# 예측값 : 
hat_y = model.predict(test_X)
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,9,1
2,0,0,10


# 3. 클래스를 생성하여 모델 생성함수 사용

In [42]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu'):
        # 모델 구성
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(50, activation=activation))
#         model.add(Dense(30, activation=activation))
#         model.add(Dense(3, activation='softmax'))
        model = Sequential([
            Input(input_dim),
            Dense(50, activation=activation),
            Dense(30, activation=activation),
            Dense(3, activation='softmax'),
        ])
        # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])
        return model

In [44]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [43]:
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 50)                250       
                                                                 
 dense_23 (Dense)            (None, 30)                1530      
                                                                 
 dense_24 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [45]:
# 3. 학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1)

Epoch 1/50
4/4 [==============================] - 0s 36ms/step - loss: 1.3997 - accuracy: 0.3241 - val_loss: 1.0384 - val_accuracy: 0.1667
Epoch 2/50
4/4 [==============================] - 0s 6ms/step - loss: 1.0416 - accuracy: 0.3426 - val_loss: 0.9367 - val_accuracy: 0.5833
Epoch 3/50
4/4 [==============================] - 0s 11ms/step - loss: 0.8885 - accuracy: 0.6111 - val_loss: 0.9429 - val_accuracy: 0.4167
Epoch 4/50
4/4 [==============================] - 0s 10ms/step - loss: 0.8434 - accuracy: 0.5093 - val_loss: 0.8622 - val_accuracy: 0.5000
Epoch 5/50
4/4 [==============================] - 0s 9ms/step - loss: 0.7568 - accuracy: 0.7130 - val_loss: 0.7449 - val_accuracy: 0.5833
Epoch 6/50
4/4 [==============================] - 0s 9ms/step - loss: 0.6740 - accuracy: 0.8148 - val_loss: 0.6681 - val_accuracy: 0.6667
Epoch 7/50
4/4 [==============================] - 0s 8ms/step - loss: 0.6241 - accuracy: 0.8704 - val_loss: 0.6225 - val_accuracy: 0.6667
Epoch 8/50
4/4 [===============

In [47]:
# 4. 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100}%')

1/1 [==============================] - 0s 17ms/step - loss: 0.1389 - accuracy: 1.0000
정확도 : 100.0%


# 4. 함수형 API
- 병렬처리, Residual block 구현

In [50]:
# 기존 모델 스타일 1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# 기존 모델 스타일 2
model = Sequential([
    Input(shape=(4,)),
    Dense(50, activation='relu'),
    Dense(30, activation='relu'),
    Dense(3, activation='softmax'),
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 50)                250       
                                                                 
 dense_32 (Dense)            (None, 30)                1530      
                                                                 
 dense_33 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [49]:
# 기존 모델 스타일 3 (함수형 API 방식)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output_ = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output_)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_28 (Dense)            (None, 50)                250       
                                                                 
 dense_29 (Dense)            (None, 30)                1530      
                                                                 
 dense_30 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [52]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output_ = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output_)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_38 (Dense)               (None, 50)           250         ['input_11[0][0]']               
                                                                                                  
 dense_39 (Dense)               (None, 80)           400         ['input_11[0][0]']               
                                                                                                  
 dense_40 (Dense)               (None, 30)           150         ['input_11[0][0]']               
                                                                                            

In [53]:
# Residual Block : 딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add #, concatenate
#from tensorflow.keras import Model
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2])
output_ = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_, outputs=output_)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_42 (Dense)               (None, 50)           250         ['input_12[0][0]']               
                                                                                                  
 dense_43 (Dense)               (None, 50)           2550        ['dense_42[0][0]']               
                                                                                                  
 add (Add)                      (None, 50)           0           ['dense_42[0][0]',               
                                                                  'dense_43[0][0]']         

In [54]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=0)

In [55]:
# 학습 평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy: {accuracy}')

accuracy: 0.9666666388511658
